# Extract Version từ Title

Script để extract cột `version` (phiên bản xe) từ cột `title` trong dữ liệu bonbanh.com

Format title: `Xe {Make} {Model} {Version} {Year} - {Price}`


In [15]:
import pandas as pd
import re
import shutil
from pathlib import Path


In [16]:
# Đường dẫn file CSV
# Thử các đường dẫn có thể có
possible_paths = [
    Path('data/toyota_cars_20251202_132900.csv'),  # Cùng thư mục với notebook
    Path('../data/toyota_cars_20251202_132900.csv'),  # Thư mục cha
    Path('./data/toyota_cars_20251202_132900.csv'),  # Cùng thư mục với notebook
]

input_file = None
data_dir = None

# Tìm file trong các đường dẫn có thể
for path in possible_paths:
    if path.exists():
        input_file = path
        data_dir = path.parent
        break

# Nếu không tìm thấy, thử tìm trong thư mục hiện tại và các thư mục con
if input_file is None:
    current_dir = Path.cwd()
    # Tìm file trong thư mục hiện tại và các thư mục con
    for path in current_dir.rglob('toyota_cars_20251202_132900.csv'):
        input_file = path
        data_dir = path.parent
        break

if input_file is None or not input_file.exists():
    print(f"❌ Error: Input file not found!")
    print(f"   Current working directory: {Path.cwd()}")
    print(f"   Searched paths:")
    for path in possible_paths:
        print(f"     - {path.absolute()} (exists: {path.exists()})")
    print(f"\n   Please check the file path or update the path in this cell.")
else:
    print(f"✅ Input file found: {input_file.absolute()}")
    
    # Định nghĩa các file output
    output_file = data_dir / 'toyota_cars_with_version.csv'
    backup_file = data_dir / 'toyota_cars_20251202_132900_backup.csv'
    
    # Tạo backup nếu chưa có
    if not backup_file.exists():
        try:
            print(f"\nCreating backup: {backup_file.absolute()}")
            shutil.copy2(input_file, backup_file)
            print("✅ Backup created successfully")
        except Exception as e:
            print(f"⚠️ Warning: Could not create backup: {e}")
    else:
        print(f"✅ Backup already exists: {backup_file.absolute()}")

    print(f"\nInput file: {input_file.absolute()}")
    print(f"Output file: {output_file.absolute()}")
    print(f"Backup file: {backup_file.absolute()}")


✅ Input file found: c:\Users\A514-54\Downloads\CarMarket-master\CarMarket-master\car-valuation-service\data\toyota_cars_20251202_132900.csv

Creating backup: c:\Users\A514-54\Downloads\CarMarket-master\CarMarket-master\car-valuation-service\data\toyota_cars_20251202_132900_backup.csv
✅ Backup created successfully

Input file: c:\Users\A514-54\Downloads\CarMarket-master\CarMarket-master\car-valuation-service\data\toyota_cars_20251202_132900.csv
Output file: c:\Users\A514-54\Downloads\CarMarket-master\CarMarket-master\car-valuation-service\data\toyota_cars_with_version.csv
Backup file: c:\Users\A514-54\Downloads\CarMarket-master\CarMarket-master\car-valuation-service\data\toyota_cars_20251202_132900_backup.csv


In [17]:
# Đọc CSV từ file gốc (không chỉnh sửa file gốc)
if input_file.exists():
    df = pd.read_csv(input_file, encoding='utf-8-sig')
    
    # Tạo bản copy để làm việc (đảm bảo không ảnh hưởng đến file gốc)
    df_work = df.copy()
    
    print(f"Total rows: {len(df_work)}")
    print(f"Columns: {df_work.columns.tolist()}")
    print(f"\n✅ Working on copy of original data")
    df_work.head()
else:
    print("❌ Please fix the input file path in cell 2 first!")


Total rows: 4502
Columns: ['ad_id', 'make', 'model', 'title', 'price_vnd', 'mileage', 'location', 'year', 'fuel', 'engine', 'gearbox', 'body', 'color', 'seats', 'engine_power', 'origin', 'accident_free', 'single_owner', 'description', 'url']

✅ Working on copy of original data


## Hàm extract version từ title


In [ ]:
def extract_version(title, make, model):
    """
    Extract version từ title
    Format: "Xe {Make} {Model} {Version} {Year} - {Price}"
    
    Examples:
    - "Xe Toyota Vios 1.5E 2017 - 255 Triệu" -> "1.5E"
    - "Xe Toyota Vios 1.5G 2020 - 395 Triệu" -> "1.5G"
    - "Xe Toyota Vios G 1.5 CVT 2021 - 420 Triệu" -> "G 1.5 CVT"
    - "Xe Toyota Vios E CVT 2025 - 428 Triệu" -> "E CVT"
    """
    if pd.isna(title) or not title:
        return None
    
    title = str(title).strip()
    
    # Loại bỏ prefix "Xe " nếu có
    if title.startswith("Xe "):
        title = title[3:]
    
    # Loại bỏ Make và Model từ đầu
    if make and model:
        make_str = str(make).strip()
        model_str = str(model).strip()
        
        # Pattern: Make Model Version Year
        pattern = rf"^{re.escape(make_str)}\s+{re.escape(model_str)}\s+(.+?)\s+(20\d{{2}})"
        match = re.search(pattern, title, re.IGNORECASE)
        
        if match:
            version = match.group(1).strip()
            # Loại bỏ các ký tự không cần thiết ở cuối
            version = re.sub(r'\s*-\s*$', '', version)  # Loại bỏ " -" ở cuối
            return version if version else None
    
    # Fallback: Tìm pattern số + chữ hoặc chữ + số trước năm
    # Pattern: ... Version 20XX ...
    pattern = r'(.+?)\s+(20\d{2})\s*-'
    match = re.search(pattern, title)
    if match:
        # Lấy phần trước năm, loại bỏ Make và Model
        before_year = match.group(1).strip()
        if make and model:
            make_model_pattern = rf"{re.escape(make)}\s+{re.escape(model)}\s+"
            before_year = re.sub(make_model_pattern, '', before_year, flags=re.IGNORECASE)
        
        version = before_year.strip()
        return version if version else None
    
    return None


In [ ]:
# Test hàm extract_version với một vài ví dụ
test_cases = [
    ("Xe Toyota Vios 1.5E 2017 - 255 Triệu", "Toyota", "Vios"),
    ("Xe Toyota Vios 1.5G 2020 - 395 Triệu", "Toyota", "Vios"),
    ("Xe Toyota Vios G 1.5 CVT 2021 - 420 Triệu", "Toyota", "Vios"),
    ("Xe Toyota Vios E CVT 2025 - 428 Triệu", "Toyota", "Vios"),
    ("Xe Toyota Camry 2.5G 2013 - 435 Triệu", "Toyota", "Camry"),
]

print("Testing extract_version function:")
print("=" * 70)
for title, make, model in test_cases:
    version = extract_version(title, make, model)
    print(f"Title: {title}")
    print(f"Version: {version}")
    print("-" * 70)


Testing extract_version function:
Title: Xe Toyota Vios 1.5E 2017 - 255 Triệu
Version: 1.5E
----------------------------------------------------------------------
Title: Xe Toyota Vios 1.5G 2020 - 395 Triệu
Version: 1.5G
----------------------------------------------------------------------
Title: Xe Toyota Vios G 1.5 CVT 2021 - 420 Triệu
Version: G 1.5 CVT
----------------------------------------------------------------------
Title: Xe Toyota Vios E CVT 2025 - 428 Triệu
Version: E CVT
----------------------------------------------------------------------
Title: Xe Toyota Camry 2.5G 2013 - 435 Triệu
Version: 2.5G
----------------------------------------------------------------------


## Apply hàm extract_version cho toàn bộ dataframe


In [ ]:
# Apply hàm extract_version trên bản copy
if 'df_work' in locals():
    df_work['version'] = df_work.apply(
        lambda row: extract_version(row['title'], row['make'], row['model']),
        axis=1
    )
    
    print(f"Total rows: {len(df_work)}")
    print(f"Rows with version extracted: {df_work['version'].notna().sum()}")
    print(f"Rows without version: {df_work['version'].isna().sum()}")
    print(f"\n✅ Version extraction completed on copy (original file unchanged)")
else:
    print("❌ Please run cell 3 first to load the data!")


Total rows: 4502
Rows with version extracted: 4456
Rows without version: 46

✅ Version extraction completed on copy (original file unchanged)


In [ ]:
# Xem một vài ví dụ
if 'df_work' in locals() and 'version' in df_work.columns:
    print("Sample rows with extracted version:")
    print("=" * 100)
    sample_df = df_work[df_work['version'].notna()][['title', 'make', 'model', 'version', 'year']].head(20)
    for idx, row in sample_df.iterrows():
        print(f"Title: {row['title']}")
        print(f"Make: {row['make']}, Model: {row['model']}, Version: {row['version']}, Year: {row['year']}")
        print("-" * 100)
else:
    print("❌ Please run previous cells first!")


Sample rows with extracted version:
Title: Xe Toyota Vios 1.5E 2017 - 255 Triệu
Make: Toyota, Model: Vios, Version: 1.5E, Year: 2017.0
----------------------------------------------------------------------------------------------------
Title: Xe Toyota Vios 1.5G 2020 - 395 Triệu
Make: Toyota, Model: Vios, Version: 1.5G, Year: 2020.0
----------------------------------------------------------------------------------------------------
Title: Xe Toyota Vios G 1.5 CVT 2021 - 420 Triệu
Make: Toyota, Model: Vios, Version: G 1.5 CVT, Year: 2021.0
----------------------------------------------------------------------------------------------------
Title: Xe Toyota Vios 1.5E 2018 - 285 Triệu
Make: Toyota, Model: Vios, Version: 1.5E, Year: 2018.0
----------------------------------------------------------------------------------------------------
Title: Xe Toyota Vios E CVT 2025 - 428 Triệu
Make: Toyota, Model: Vios, Version: E CVT, Year: 2025.0
-----------------------------------------------------

In [ ]:
# Xem các rows không extract được version
if 'df_work' in locals() and 'version' in df_work.columns:
    if df_work['version'].isna().sum() > 0:
        print("Rows without version (first 10):")
        print("=" * 100)
        no_version_df = df_work[df_work['version'].isna()][['title', 'make', 'model', 'year']].head(10)
        for idx, row in no_version_df.iterrows():
            print(f"Title: {row['title']}")
            print(f"Make: {row['make']}, Model: {row['model']}, Year: {row['year']}")
            print("-" * 100)
    else:
        print("✅ All rows have version extracted!")
else:
    print("❌ Please run previous cells first!")


Rows without version (first 10):
Title: nan
Make: Toyota, Model: innova, Year: nan
----------------------------------------------------------------------------------------------------
Title: Xe Toyota Camry LE 2.2 AT 1995 - 75 Triệu
Make: Toyota, Model: Camry, Year: nan
----------------------------------------------------------------------------------------------------
Title: Xe Toyota Camry 2.0 MT Trước 1990 - 139 Triệu
Make: Toyota, Model: Camry, Year: nan
----------------------------------------------------------------------------------------------------
Title: Xe Toyota Camry LE 2.0 AT 1995 - 75 Triệu
Make: Toyota, Model: Camry, Year: nan
----------------------------------------------------------------------------------------------------
Title: Xe Toyota Camry 1.8 MT Trước 1990 - 32 Triệu
Make: Toyota, Model: Camry, Year: nan
----------------------------------------------------------------------------------------------------
Title: Xe Toyota Camry XLi 2.2 1998 - 95 Triệu
Make: Toyo

In [ ]:
# Thống kê các version phổ biến
if 'df_work' in locals() and 'version' in df_work.columns:
    if df_work['version'].notna().sum() > 0:
        print("Top 20 most common versions:")
        print("=" * 50)
        version_counts = df_work['version'].value_counts().head(20)
        for version, count in version_counts.items():
            print(f"{version:30s}: {count:4d} cars")
    else:
        print("No versions extracted yet.")
else:
    print("❌ Please run previous cells first!")


Top 20 most common versions:
Cruiser 3.5 V6                :  272 cars
2.0E                          :  205 cars
Cross 1.8V                    :  193 cars
VX 2.7L                       :  152 cars
1.5G                          :  152 cars
G 1.5 CVT                     :  138 cars
altis 1.8G AT                 :  124 cars
G                             :  123 cars
2.5Q                          :  117 cars
2.7V 4x2 AT                   :  115 cars
Cross Top 1.5 CVT             :   98 cars
G 1.0 CVT                     :   95 cars
1.5E                          :   94 cars
2.4L 4x2 AT                   :   93 cars
E CVT                         :   87 cars
Cross 1.5 D-CVT               :   87 cars
2.5G                          :   76 cars
2.0Q                          :   75 cars
2.0G                          :   74 cars
Cross 1.5 CVT                 :   71 cars


## Reorder columns và lưu file mới


In [24]:
# Reorder columns: đặt version sau model
if 'df_work' in locals() and 'version' in df_work.columns:
    cols = df_work.columns.tolist()
    
    # Tìm vị trí của 'model'
    if 'model' in cols:
        model_idx = cols.index('model')
        
        # Di chuyển 'version' ngay sau 'model'
        if 'version' in cols:
            cols.remove('version')
            cols.insert(model_idx + 1, 'version')
        
        df_work = df_work[cols]
        
        print("Column order:")
        print(cols)
    else:
        print("❌ 'model' column not found!")
else:
    print("❌ Please run previous cells first!")


Column order:
['ad_id', 'make', 'model', 'version', 'title', 'price_vnd', 'mileage', 'location', 'year', 'fuel', 'engine', 'gearbox', 'body', 'color', 'seats', 'engine_power', 'origin', 'accident_free', 'single_owner', 'description', 'url']


In [25]:
# Lưu file mới (không ghi đè file gốc)
if 'df_work' in locals() and 'version' in df_work.columns:
    try:
        df_work.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"✅ Saved to: {output_file}")
        print(f"Total rows: {len(df_work)}")
        print(f"Rows with version: {df_work['version'].notna().sum()} ({df_work['version'].notna().sum()/len(df_work)*100:.1f}%)")
        print(f"\n✅ Original file unchanged: {input_file}")
        if backup_file.exists():
            print(f"✅ Backup file: {backup_file}")
    except Exception as e:
        print(f"❌ Error saving file: {e}")
else:
    print("❌ Please run previous cells first!")


✅ Saved to: data\toyota_cars_with_version.csv
Total rows: 4502
Rows with version: 4456 (99.0%)

✅ Original file unchanged: data\toyota_cars_20251202_132900.csv
✅ Backup file: data\toyota_cars_20251202_132900_backup.csv


# Extract Version từ Title

Script để extract cột `version` (phiên bản xe) từ cột `title` trong dữ liệu bonbanh.com

Format title: `Xe {Make} {Model} {Version} {Year} - {Price}`


In [10]:
import pandas as pd
import re
from pathlib import Path


In [12]:
# Đường dẫn file CSV
input_file = Path(r'C:\Users\A514-54\Downloads\CarMarket-master\CarMarket-master\car-valuation-service\data\toyota_cars_20251202_132900.csv')
output_file = Path('../data/toyota_cars_with_version.csv')

# Tạo bản copy của file gốc để đảm bảo an toàn
import shutil
backup_file = Path('../data/toyota_cars_20251202_132900_backup.csv')
if not backup_file.exists():
    print(f"Creating backup: {backup_file}")
    shutil.copy2(input_file, backup_file)
    print("✅ Backup created successfully")
else:
    print(f"✅ Backup already exists: {backup_file}")

print(f"\nInput file: {input_file}")
print(f"Output file: {output_file}")
print(f"Backup file: {backup_file}")


Creating backup: ..\data\toyota_cars_20251202_132900_backup.csv


FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\toyota_cars_20251202_132900_backup.csv'

In [ ]:
# Đọc CSV từ file gốc (không chỉnh sửa file gốc)
df = pd.read_csv(input_file, encoding='utf-8-sig')

# Tạo bản copy để làm việc (đảm bảo không ảnh hưởng đến file gốc)
df_work = df.copy()

print(f"Total rows: {len(df_work)}")
print(f"Columns: {df_work.columns.tolist()}")
print(f"\n✅ Working on copy of original data")
df_work.head()


Total rows: 4502
Columns: ['ad_id', 'make', 'model', 'title', 'price_vnd', 'mileage', 'location', 'year', 'fuel', 'engine', 'gearbox', 'body', 'color', 'seats', 'engine_power', 'origin', 'accident_free', 'single_owner', 'description', 'url']


,ad_id,make,model,title,price_vnd,mileage,location,year,fuel,engine,gearbox,body,color,seats,engine_power,origin,accident_free,single_owner,description,url
0,xe-toyota-vios-1.5e-2017-6507775,Toyota,Vios,Xe Toyota Vios 1.5E 2017 - 255 Triệu,255.0,110000 km,316 Nguyễn Xiển - Đại kim - Hoàng Mai Hà Nội,2017.0,Xăng,Xăng 1.5 L,Số sàn,Sedan,Bạc,5.0,NaN,Việt Nam,True,False,NaN,https://bonbanh.com/xe-toyota-vios-1.5e-2017-6...
1,xe-toyota-vios-1.5g-2020-6297971,Toyota,Vios,Xe Toyota Vios 1.5G 2020 - 395 Triệu,395.0,108000 km,TP. Buôn Ma Thuột Đăk Lăk,2020.0,Xăng,Xăng 1.5 L,Tự động,Sedan,Ghi,5.0,NaN,Việt Nam,NaN,False,NaN,https://bonbanh.com/xe-toyota-vios-1.5g-2020-6...
2,xe-toyota-vios-g-1.5-cvt-2021-6391460,Toyota,Vios,Xe Toyota Vios G 1.5 CVT 2021 - 420 Triệu,420.0,60000 km,Hoài Đức Hà Nội,2021.0,Xăng,Xăng 1.5 L,Tự động,Sedan,Cát,5.0,NaN,Việt Nam,True,False,NaN,https://bonbanh.com/xe-toyota-vios-g-1.5-cvt-2...
3,xe-toyota-vios-1.5e-2018-6535714,Toyota,Vios,Xe Toyota Vios 1.5E 2018 - 285 Triệu,285.0,100000 km,huyện Quỳnh Phụ Thái Bình,2018.0,Xăng,Xăng 1.5 L,Số sàn,Sedan,Cát,5.0,NaN,Việt Nam,False,False,NaN,https://bonbanh.com/xe-toyota-vios-1.5e-2018-6...
4,xe-toyota-vios-e-cvt-2025-6514902,Toyota,Vios,Xe Toyota Vios E CVT 2025 - 428 Triệu,428.0,NaN,315 Trường Chinh - Thanh Xuân Hà Nội,2025.0,Xăng,Xăng 1.5 L,Tự động,Sedan,Đen,5.0,NaN,Việt Nam,NaN,False,NaN,https://bonbanh.com/xe-toyota-vios-e-cvt-2025-...


## Hàm extract version từ title


In [ ]:
def extract_version(title, make, model):
    """
    Extract version từ title
    Format: "Xe {Make} {Model} {Version} {Year} - {Price}"
    
    Examples:
    - "Xe Toyota Vios 1.5E 2017 - 255 Triệu" -> "1.5E"
    - "Xe Toyota Vios 1.5G 2020 - 395 Triệu" -> "1.5G"
    - "Xe Toyota Vios G 1.5 CVT 2021 - 420 Triệu" -> "G 1.5 CVT"
    - "Xe Toyota Vios E CVT 2025 - 428 Triệu" -> "E CVT"
    """
    if pd.isna(title) or not title:
        return None
    
    title = str(title).strip()
    
    # Loại bỏ prefix "Xe " nếu có
    if title.startswith("Xe "):
        title = title[3:]
    
    # Loại bỏ Make và Model từ đầu
    if make and model:
        make_str = str(make).strip()
        model_str = str(model).strip()
        
        # Pattern: Make Model Version Year
        pattern = rf"^{re.escape(make_str)}\s+{re.escape(model_str)}\s+(.+?)\s+(20\d{{2}})"
        match = re.search(pattern, title, re.IGNORECASE)
        
        if match:
            version = match.group(1).strip()
            # Loại bỏ các ký tự không cần thiết ở cuối
            version = re.sub(r'\s*-\s*$', '', version)  # Loại bỏ " -" ở cuối
            return version if version else None
    
    # Fallback: Tìm pattern số + chữ hoặc chữ + số trước năm
    # Pattern: ... Version 20XX ...
    pattern = r'(.+?)\s+(20\d{2})\s*-'
    match = re.search(pattern, title)
    if match:
        # Lấy phần trước năm, loại bỏ Make và Model
        before_year = match.group(1).strip()
        if make and model:
            make_model_pattern = rf"{re.escape(make)}\s+{re.escape(model)}\s+"
            before_year = re.sub(make_model_pattern, '', before_year, flags=re.IGNORECASE)
        
        version = before_year.strip()
        return version if version else None
    
    return None


In [8]:
# Test hàm extract_version với một vài ví dụ
test_cases = [
    ("Xe Toyota Vios 1.5E 2017 - 255 Triệu", "Toyota", "Vios"),
    ("Xe Toyota Vios 1.5G 2020 - 395 Triệu", "Toyota", "Vios"),
    ("Xe Toyota Vios G 1.5 CVT 2021 - 420 Triệu", "Toyota", "Vios"),
    ("Xe Toyota Vios E CVT 2025 - 428 Triệu", "Toyota", "Vios"),
    ("Xe Toyota Camry 2.5G 2013 - 435 Triệu", "Toyota", "Camry"),
]

print("Testing extract_version function:")
print("=" * 70)
for title, make, model in test_cases:
    version = extract_version(title, make, model)
    print(f"Title: {title}")
    print(f"Version: {version}")
    print("-" * 70)


Testing extract_version function:
Title: Xe Toyota Vios 1.5E 2017 - 255 Triệu
Version: 1.5E
----------------------------------------------------------------------
Title: Xe Toyota Vios 1.5G 2020 - 395 Triệu
Version: 1.5G
----------------------------------------------------------------------
Title: Xe Toyota Vios G 1.5 CVT 2021 - 420 Triệu
Version: G 1.5 CVT
----------------------------------------------------------------------
Title: Xe Toyota Vios E CVT 2025 - 428 Triệu
Version: E CVT
----------------------------------------------------------------------
Title: Xe Toyota Camry 2.5G 2013 - 435 Triệu
Version: 2.5G
----------------------------------------------------------------------


## Apply hàm extract_version cho toàn bộ dataframe


In [ ]:
# Apply hàm extract_version trên bản copy
df_work['version'] = df_work.apply(
    lambda row: extract_version(row['title'], row['make'], row['model']),
    axis=1
)

print(f"Total rows: {len(df_work)}")
print(f"Rows with version extracted: {df_work['version'].notna().sum()}")
print(f"Rows without version: {df_work['version'].isna().sum()}")
print(f"\n✅ Version extraction completed on copy (original file unchanged)")


In [ ]:
# Xem một vài ví dụ
print("Sample rows with extracted version:")
print("=" * 100)
sample_df = df_work[df_work['version'].notna()][['title', 'make', 'model', 'version', 'year']].head(20)
for idx, row in sample_df.iterrows():
    print(f"Title: {row['title']}")
    print(f"Make: {row['make']}, Model: {row['model']}, Version: {row['version']}, Year: {row['year']}")
    print("-" * 100)


In [ ]:
# Xem các rows không extract được version
if df_work['version'].isna().sum() > 0:
    print("Rows without version (first 10):")
    print("=" * 100)
    no_version_df = df_work[df_work['version'].isna()][['title', 'make', 'model', 'year']].head(10)
    for idx, row in no_version_df.iterrows():
        print(f"Title: {row['title']}")
        print(f"Make: {row['make']}, Model: {row['model']}, Year: {row['year']}")
        print("-" * 100)


In [ ]:
# Thống kê các version phổ biến
if df_work['version'].notna().sum() > 0:
    print("Top 20 most common versions:")
    print("=" * 50)
    version_counts = df_work['version'].value_counts().head(20)
    for version, count in version_counts.items():
        print(f"{version:30s}: {count:4d} cars")


## Reorder columns và lưu file mới


In [ ]:
# Reorder columns: đặt version sau model
cols = df_work.columns.tolist()

# Tìm vị trí của 'model'
model_idx = cols.index('model')

# Di chuyển 'version' ngay sau 'model'
if 'version' in cols:
    cols.remove('version')
    cols.insert(model_idx + 1, 'version')

df_work = df_work[cols]

print("Column order:")
print(cols)


In [ ]:
# Lưu file mới (không ghi đè file gốc)
df_work.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"✅ Saved to: {output_file}")
print(f"Total rows: {len(df_work)}")
print(f"Rows with version: {df_work['version'].notna().sum()} ({df_work['version'].notna().sum()/len(df_work)*100:.1f}%)")
print(f"\n✅ Original file unchanged: {input_file}")
print(f"✅ Backup file: {backup_file}")
